In [642]:
import numpy as np

# multi-asset Geometric Brownian motion
class multiGeometric:
    
    def __init__(self, s0 = None, T = None, N = None, cov = None):
        self.s0 = s0
        self.T = T
        self.N = N
        self.cov = cov
        
    # function that generates a specific process
    def gen_process(self):
        
        dt = np.divide(self.T,self.N)
        sigma = np.zeros(2)
        processes = np.zeros((self.N,2))
        processes[0,:] = np.log(self.s0)
        
        drift = np.zeros(1)
        diffusion = np.zeros(1)
        
        for i in range(1,self.N):
            z1 = np.random.normal(0,1)
            sigma[0] = cov[0,0]
            sigma[1] = cov[1,1]
            
            drift = 0.5*(sigma**2)*dt
            diffusion = np.sqrt(self.T/self.N)*np.matmul(np.linalg.cholesky(self.cov), np.random.normal(0,1,size=2))
            
            processes[i,:] = processes[i-1,:] - drift + diffusion
        
        return np.exp(processes)
    
    # function that loops through the afordefined to generate multiple paths
    def gen_path(self, nbPaths = None):
        
        dt = np.divide(self.T,self.N)
        dualPaths = np.zeros(((nbPaths,self.N,2)))
        
        for i in range(nbPaths):
            dualPaths[i,:,:] = self.gen_process()
        
        return dualPaths
    
s0 = np.array([100,100])
cov = np.array([[0.1, 0.05],[0.05, 0.2]])
    
sto = multiGeometric(s0 = s0, T = 1, N = 30, cov = cov)
spot = sto.gen_path(nbPaths = 3)
path = sto.gen_process()
# working fine

In [23]:
from math import log, sqrt, exp, pi
import numpy as np
import scipy.stats as stats

def bivnormcdf(a,b,rho):

	if a <= 0. and b <= 0. and rho <= 0. :
		aprime = a/sqrt(2.*(1.-rho**2.))
		bprime = b/sqrt(2.*(1.-rho**2.))
		A = np.array([0.3253030, 0.4211071, 0.1334425, 0.006374323])
		B = np.array([0.1337764, 0.6243247, 1.3425378, 2.2626645])

		t = 0.

		for i in range(4):
			for j in range(4):
				x = B[i]
				y = B[j]
				t += A[i]*A[j]* exp(aprime*(2.*x - aprime) \
				     + (bprime*(2.*y - bprime)) + (2.*rho * (x - aprime)*(y-bprime)))

		p = (sqrt(1.-rho**2.)/pi) * t

	elif a * b * rho <= 0. :
		if a <= 0. and b >= 0. and rho >= 0. :
			p = stats.norm.cdf(a) - bivnormcdf(a,-b,-rho)
		elif a >= 0. and b <= 0. and rho >= 0. :
			p = stats.norm.cdf(b) - bivnormcdf(-a,b,-rho)
		elif a >= 0. and b >= 0. and rho <= 0. :
			p = stats.norm.cdf(a) + stats.norm.cdf(b) - 1. + bivnormcdf(-a,-b,rho)

	elif a*b*rho > 0. :
		if a >= 0. :
			asign = 1.
		else:
			asign = -1.

		if b >= 0.:
			bsign = 1.
		else:
			bsign = -1.

		rho1 = (rho*a - b)*asign/(sqrt(a**2. - (2.*rho*a*b) + b**2.))
		rho2 = (rho*b - a)*bsign/(sqrt(a**2. - (2.*rho*a*b) + b**2.))
		delta = (1. - (asign*bsign))/4.

		p = bivnormcdf(a,0,rho1) + bivnormcdf(b,0,rho2) - delta
        
	return p

In [616]:
import numpy as np
import scipy.stats as stats

class EuropeanRainbow:
    def __init__(self):
        pass

    def get_Stulz_price(self, S=None, cov=None, r=None, K=None, matu=None, N=None, nbPaths=None):

        dt = matu / N
        T = np.arange(0, N) * dt
        T = np.repeat(np.flip(T[None, :]), S.shape[0], axis=0)

        spot1 = np.zeros((nbPaths, N))
        spot2 = np.zeros((nbPaths, N))

        eta1 = np.zeros((nbPaths, N))
        eta2 = np.zeros((nbPaths, N))

        beta1 = np.zeros((nbPaths, N))
        beta2 = np.zeros((nbPaths, N))

        gamma1 = np.zeros((nbPaths, N))
        gamma2 = np.zeros((nbPaths, N))

        spot1[:, :] = S[:, :, 0]
        spot2[:, :] = S[:, :, 1]

        sigma1 = np.sqrt(cov[0, 0])
        sigma2 = np.sqrt(cov[1, 1])
        rho = cov[0, 1] / (sigma1 * sigma2)

        sig = cov[0,0] + cov[1,1] - 2 * cov[0,1]
        rho_indiv1 = (rho * sigma2 - sigma1) / np.sqrt(sig)
        rho_indiv2 = (rho * sigma1 - sigma2) / np.sqrt(sig)

        price = np.zeros((nbPaths, N-1))

        with np.errstate(divide="ignore"):
            eta1 = np.divide(np.log(spot1 / K) + (r + 0.5 * (sigma1 ** 2)) * T, (sigma1 * np.sqrt(T)))
            eta2 = np.divide(np.log(spot2 / K) + (r + 0.5 * (sigma2 ** 2)) * T, (sigma2 * np.sqrt(T)))

            beta1 = np.divide(np.log(spot2 / spot1) - 0.5 * ((sig ** 2) * np.sqrt(T)), np.sqrt(sig) * np.sqrt(T))
            beta2 = np.divide(np.log(spot1 / spot2) - 0.5 * ((sig ** 2) * np.sqrt(T)), np.sqrt(sig) * np.sqrt(T))

            gamma1 = eta1 - sigma1 * np.sqrt(T)
            gamma2 = eta2 - sigma2 * np.sqrt(T)

        for i in range(0,N-1):

            for j in range(0,nbPaths):

                price[j,i] = spot2[j,i] * bivnormcdf(eta2[j,i],beta2[j,i],rho_indiv2) + \
                             spot1[j,i] * bivnormcdf(eta1[j,i],beta1[j,i],rho_indiv1) - \
                             K * np.exp(-r * T[j,i]) * bivnormcdf(gamma2[j,i],gamma1[j,i],rho)
                
                if price[j,i] < 0:
                    price[j,i] = 0

        return price

    def get_Stulz_delta(self, S=None, cov=None, r=None, K=None, matu=None, N=None, nbPaths=None):

        dt = matu / N
        T = np.arange(0, N) * dt
        T = np.repeat(np.flip(T[None, :]), S.shape[0], axis=0)

        spot1 = np.zeros((nbPaths, N))
        spot2 = np.zeros((nbPaths, N))

        eta1 = np.zeros((nbPaths, N))
        eta2 = np.zeros((nbPaths, N))

        beta1 = np.zeros((nbPaths, N))
        beta2 = np.zeros((nbPaths, N))

        gamma1 = np.zeros((nbPaths, N))
        gamma2 = np.zeros((nbPaths, N))

        spot1[:, :] = S[:, :, 0]
        spot2[:, :] = S[:, :, 1]

        sigma1 = np.sqrt(cov[0, 0])
        sigma2 = np.sqrt(cov[1, 1])
        rho = cov[0, 1] / (sigma1 * sigma2)

        sig = cov[0,0] + cov[1,1] - 2 * cov[0,1]
        rho_indiv1 = (rho * sigma2 - sigma1) / np.sqrt(sig)
        rho_indiv2 = (rho * sigma1 - sigma2) / np.sqrt(sig)

        delta1 = np.zeros((nbPaths, N-1))
        delta2 = np.zeros((nbPaths, N-1))

        with np.errstate(divide="ignore"):
            eta1 = np.divide(np.log(spot1 / K) + (r + 0.5 * (sigma1 ** 2)) * T, (sigma1 * np.sqrt(T)))
            eta2 = np.divide(np.log(spot2 / K) + (r + 0.5 * (sigma2 ** 2)) * T, (sigma2 * np.sqrt(T)))

            beta1 = np.divide(np.log(spot2 / spot1) - 0.5 * ((sig ** 2) * np.sqrt(T)), np.sqrt(sig) * np.sqrt(T))
            beta2 = np.divide(np.log(spot1 / spot2) - 0.5 * ((sig ** 2) * np.sqrt(T)), np.sqrt(sig) * np.sqrt(T))

            gamma1 = eta1 - sigma1 * np.sqrt(T)
            gamma2 = eta2 - sigma2 * np.sqrt(T)

        for i in range(0,N-1):

            for j in range(0, nbPaths):

                delta1[j,i] = bivnormcdf(eta1[j,i],beta1[j,i],rho_indiv1)+ \
                         stats.norm.cdf(np.divide(eta1[j,i]-rho_indiv1*beta1[j,i],np.sqrt(1-rho_indiv1**2)))* \
                         np.exp(-0.5*beta1[j,i]*beta1[j,i])*(1/np.sqrt(np.min(np.log((spot1),np.log(spot2)))))- \
                         np.divide(spot2[j,i],spot1[j,i])*stats.norm.cdf(np.divide(eta2[j,i]-rho_indiv2*beta2[j,i], \
                         np.sqrt(1-rho_indiv2**2)))*np.exp(-0.5*beta2[j,i]*beta2[j,i])* \
                         (1 / np.sqrt(np.min(np.log((spot1), np.log(spot2)))))

                delta2[j,i] = bivnormcdf(eta2[j,i],beta2[j,i],rho_indiv2)+ \
                         stats.norm.cdf(np.divide(eta2[j,i]-rho_indiv2*beta2[j,i],np.sqrt(1-rho_indiv2**2)))* \
                         np.exp(-0.5*beta1[j,i]*beta1[j,i])*(1/np.sqrt(np.min(np.log((spot1),np.log(spot2)))))- \
                         np.divide(spot1[j,i],spot2[j,i])*stats.norm.cdf(np.divide(eta1[j,i]-rho_indiv1*beta1[j,i], \
                         np.sqrt(1-rho_indiv1**2)))*np.exp(-0.5*beta2[j,i]*beta2[j,i])* \
                         (1 / np.sqrt(np.min(np.log((spot1), np.log(spot2)))))
                
        delta = np.zeros(((2, nbPaths, N-1)))
        delta[0,:,:] = delta1[:,:]
        delta[1,:,:] = delta2[:,:]

        return delta
    
    def get_Stulz_PnL(self, S = None, payoff = None, delta = None, matu = None, r = None, \
                      final_period_cost = None, eps = None, N = None):
        
        dt = np.divide(matu,N)
        
        # we compute the initial PnL
        PnL_Stulz = np.multiply(S[:,0,0], - delta[0,:,0]) + np.multiply(S[:,0,1], - delta[1,:,0])
        PnL_Stulz = PnL_Stulz - np.abs(delta[0,:,0])*S[:,0,0]*eps - np.abs(delta[1,:,0])*S[:,0,1]*eps
        PnL_Stulz = PnL_Stulz * np.exp(r * dt)
        
        # we compute the PnL at each in-between time steps
        for t in range(1, N-1):
            PnL_Stulz = PnL_Stulz + np.multiply(S[:,t,0], -delta[0,:,t] + delta[0,:,t-1]) + \
                        np.multiply(S[:,t,1], -delta[1,:,t] + delta[1,:,t-1])
            
            PnL_Stulz = PnL_Stulz - np.abs(delta[0,:,t] - delta[0,:,t-1]) * S[:,t,0] * eps \
                        - np.abs(delta[1,:,t] - delta[1,:,t-1]) * S[:,t,1] * eps
            
            PnL_Stulz = PnL_Stulz * np.exp(r * dt)
        
        # we compute the final PnL
        PnL_Stulz = PnL_Stulz + np.multiply(S[:,N-1,0], delta[0,:,N-2]) + np.multiply(S[:,N-1,1], delta[1,:,N-2]) + payoff
        
        if final_period_cost:
            PnL_Stulz = PnL_Stulz - np.abs(delta[0,:,N-1]) * S[:,N-1,0] * eps - np.abs(delta[1,:,N-1]) * S[:,N-1,1] * eps
            
        return PnL_Stulz

In [664]:
m = np.array([0.1, 0.1])
spot_init = np.array([100, 100])
T = 1
N = 30
cov = np.array([[0.1, 0.05], [0.05, 0.1]])
prob1 = 0.3
prob2 = 0.3
r = 0.0
K = 100
matu = 1
nbPaths = 1
eps=0.0
payoff_func = lambda x, y : -np.maximum(np.minimum(x-K, y-K), 0.0)
s
test = multiGeometric(s0=spot_init, T=T, N=N, cov=cov)
S = test.gen_path(nbPaths=nbPaths)
payoff_final = payoff_func(S[:,-1,0],S[:,-1,1])

rainbow = EuropeanRainbow()
price = rainbow.get_Stulz_price(S=S, cov=cov, r=r, K=K, matu=matu, N=N, nbPaths=nbPaths)
delta = rainbow.get_Stulz_delta(S=S, cov=cov, r=r, K=K, matu=matu, N=N, nbPaths=nbPaths)

# on a réussi à correctement pricer une option rainbow
# il va nous rester cet aprem à étudier la fonction PnL
# fonction codée pour le PnL, peut-être un poil chelou qu'on ait que des valeurs négatives mais wtv

pnl = rainbow.get_Stulz_PnL(S=S, delta=delta, matu=matu, N=N,eps=eps, r=r, payoff=payoff_final,final_period_cost=False)
print(pnl)

[-2.37330845]
